<a href="https://colab.research.google.com/github/GabeMaldonado/JupyterNotebooks/blob/master/cdQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# install cdQa
pip install cdqa

     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 133kB 18.4MB/s 
     |████████████████████████████████| 655kB 40.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/80/ef/b4/6b5d67b6baee6ece696e72b4092ca7f0b3aaeec2e084f61bf2
  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built cdqa tika wget regex


In [7]:
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.download import download_model, download_bnpp_data
from cdqa.pipeline.cdqa_sklearn import QAPipeline


/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [11]:
# Download Data and Models

download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='./models')


bnpp_newsroom-v1.1.csv already downloaded

bert_qa_vCPU-sklearn.joblib already downloaded


In [12]:
# Load and filter data / preprocess docs

df = pd.read_csv('data/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df.head()

,date,title,category,link,abstract,paragraphs
0,13.05.2019,The banking jobs : Assistant Vice President – ...,Careers,https://group.bnpparibas/en/news/banking-jobs-...,Within the Group’s Corporate and Institutional...,[I manage a team in charge of designing and im...
1,13.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[With François Hollande, Chairman of French fo..."
2,13.05.2019,"""The bank with an IT budget of more than EUR6 ...",Group,https://group.bnpparibas/en/news/the-bank-budg...,"Interview with Jean-Laurent Bonnafé, Director ...","[We did the groundwork between 2012 and 2016, ..."
3,10.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[As part of the ‘United Tech of Europe’ theme,..."
4,10.05.2019,When Artificial Intelligence participates in r...,Careers,https://group.bnpparibas/en/news/artificial-in...,As the competition to attract talent intensifi...,[Online recruitment is already the norm. Accor...


In [22]:
df.tail()

,date,title,category,link,abstract,paragraphs,content
1322,05.04.2002,BNP Paribas title sponsor of the Davis Cup,Press release,https://group.bnpparibas/en/press-release/bnp-...,NaN,"[BNP Paribas, one of the leading European bank...","BNP Paribas, one of the leading European banks..."
1323,05.04.2002,BNP Paribas and the French Tennis Federation r...,Press release,https://group.bnpparibas/en/press-release/bnp-...,NaN,[BNP Paribas is also continuing its rewarding ...,BNP Paribas is also continuing its rewarding r...
1324,08.02.2002,BNP Paribas takes over as Title Sponsor,Press release,https://group.bnpparibas/en/press-release/bnp-...,NaN,"[BNP Paribas, one of the leading European bank...","BNP Paribas, one of the leading European banks..."
1325,17.11.2001,"BNP Paribas brings its ""euro angels"" to Paris ...",Press release,https://group.bnpparibas/en/press-release/bnp-...,NaN,[The introduction of the euro will be a histor...,The introduction of the euro will be a histori...
1326,28.05.2001,"BNP Paribas, a leading sponsor of tennis, unve...",Press release,https://group.bnpparibas/en/press-release/bnp-...,NaN,"[For close to 28 years, BNP, and now BNP Parib...","For close to 28 years, BNP, and now BNP Pariba..."


In [0]:
# Load QAPipeline with CPU versoin of BERT Reader pretrained on SQUAD 1.1
cdqa_pipeline = QAPipeline(reader='models/bert_qa_vCPU-sklearn.joblib')

In [16]:
# Fit the retriever to the list of documents in the dataframe

cdqa_pipeline.fit(X = df)

QAPipeline(reader=BertQA(bert_model='bert-base-uncased', do_lower_case=True,
                         fp16=False, gradient_accumulation_steps=1,
                         learning_rate=3e-05, local_rank=-1, loss_scale=0,
                         max_answer_length=30, n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=2,
                         output_dir='models', predict_batch_size=8, seed=42,
                         server_ip='', server_port='', train_batch_size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1))

In [17]:
# Send a question to the pipeline and get prediction
query = 'When was the Excellence Program of BNP Paribas created'
prediction = cdqa_pipeline.predict(X = query)

3it [00:00, 809.09it/s]
100%|██████████| 231508/231508 [00:00<00:00, 934276.38B/s]


In [18]:
print('query: {}\n'.format(query))
print('answer:{}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: When was the Excellence Program of BNP Paribas created

answer:January 2016

title: BNP Paribas’ commitment to universities and schools

paragraph: Since January 2016, BNP Paribas has offered an Excellence Program targeting new Master’s level graduates (BAC+5) who show high potential. The aid program lasts 18 months and comprises three assignments of six months each. It serves as a strong career accelerator that enables participants to access high-level management positions at a faster rate. The program allows participants to discover the BNP Paribas Group and its various entities in France and abroad, build an internal and external network by working on different assignments and receive personalized assistance from a mentor and coaching firm at every step along the way.



In [19]:
# Question # 2
query2 = 'What is the Excellence Program of BNP Paribas'
prediction = cdqa_pipeline.predict(X = query2)
print('query2: {}\n'.format(query2))
print('answer:{}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

3it [00:00, 1213.40it/s]


query2: What is the Excellence Program of BNP Paribas

answer:academic partnerships

title: BNP Paribas’ commitment to universities and schools

paragraph: This large-scale project will further expand in coming years, notably internationally. Gilles Deschanel, in charge of academic partnerships at BNP Paribas CIB, observes that “this partnership aims to become one of the leading research bodies in this immensely disruptive technology. It will also play a role in transforming the financial sector through publications and major events like VivaTech.”



In [25]:
#@title
# Question # 3
query3 = 'How long is the aid program?'
prediction = cdqa_pipeline.predict(X = query3)
print('query3 {}\n'.format(query3))
print('answer:{}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

3it [00:00, 1596.82it/s]


query3 How long is the aid program?

answer:18 months

title: BNP Paribas’ commitment to universities and schools

paragraph: Since January 2016, BNP Paribas has offered an Excellence Program targeting new Master’s level graduates (BAC+5) who show high potential. The aid program lasts 18 months and comprises three assignments of six months each. It serves as a strong career accelerator that enables participants to access high-level management positions at a faster rate. The program allows participants to discover the BNP Paribas Group and its various entities in France and abroad, build an internal and external network by working on different assignments and receive personalized assistance from a mentor and coaching firm at every step along the way.

